In [2]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [3]:
NUM_CLASSES = 4
NUM_FEATURES = 2
RANDOM_SEED = 42

X_blob, y_blob = make_blobs(n_samples=500,
                             n_features=NUM_FEATURES,
                             centers=NUM_CLASSES,
                             cluster_std=1.75,
                             random_state=RANDOM_SEED)
X_blob[:10], y_blob[:10]

(array([[ -6.82709944,   6.09296417],
        [-11.59917882,  -6.97512561],
        [  8.47203397,   0.55864768],
        [ -3.19538689,   6.45313497],
        [ -4.77750283,  -7.5942415 ],
        [ -5.42653951,  -8.37825629],
        [ -4.35817981,   7.15541809],
        [ -6.40632775,  -6.96802628],
        [  2.77154308,   1.72400559],
        [ -7.9730597 ,  -6.83445026]]),
 array([3, 2, 1, 0, 2, 2, 3, 2, 1, 2]))

In [4]:
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.float)
X_blob_train, X_blob_test, y_blob_train, y_blob_test = train_test_split(X_blob ,y_blob, test_size=0.2,random_state=RANDOM_SEED)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
class BlobModel(torch.nn.Module):
    def __init__(self, input_features, output_features, hidden_units = 8):
        super().__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_features, out_features=hidden_units),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_units, out_features=hidden_units),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=hidden_units, out_features=output_features)
        )
    def forward(self, x):
        return self.sequential(x)

In [21]:
model = BlobModel(input_features=2, output_features=4, hidden_units=8).to(device)
model

BlobModel(
  (sequential): Sequential(
    (0): Linear(in_features=2, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=4, bias=True)
  )
)

In [22]:
loss_fun = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [23]:
model.eval()
with torch.inference_mode():
    y_logits = model(X_blob_test.to(device))
y_logits[:10]

tensor([[ 0.1719,  0.5843,  0.0645,  0.3154],
        [ 0.1541,  0.5453,  0.0554,  0.1250],
        [ 0.2179,  0.6700,  0.1277,  0.0941],
        [ 0.1993,  0.6434,  0.1123,  0.1155],
        [ 0.1491,  0.5527,  0.0598,  0.1487],
        [-0.2257, -0.0933, -0.1788, -0.0639],
        [-0.3678,  0.2216,  0.0577,  0.2256],
        [ 0.2315,  0.7564,  0.1721,  0.2488],
        [-0.3164,  0.1743, -0.0076,  0.2635],
        [-0.2921,  0.1408, -0.1086,  0.2611]], device='cuda:0')

In [30]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

EPOCHS = 100
X_blob_train, X_blob_test = X_blob_train.to(device), X_blob_test.to(device)
y_blob_train, y_blob_test = y_blob_train.to(device), y_blob_test.to(device)

for epoch in range(EPOCHS):
    model.train()
    y_logits = model(X_blob_train).to(device)
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

    #1 loss = loss_fun(y_logits, y_blob_train) wont work because of dtype issue, Float is not compatble here
    #2 looks like .y_blob_train.type(...) puts the new device in CPU. Shift it back to CUDA

    loss = loss_fun(y_logits, y_blob_train.type(torch.LongTensor).to(device))
    acc = torch.eq(y_pred, y_blob_train).sum() / len(y_blob_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        test_logits = model(X_blob_test).to(device)
        test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
        test_loss = loss_fun(test_logits, y_blob_test.type(torch.LongTensor).to(device))
        test_acc = torch.eq(test_pred, y_blob_test).sum() / len(y_blob_test)
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {acc:.2f}% | Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}%")


Epoch: 0 | Loss: 1.18083 | Acc: 0.11% | Test Loss: 1.07632 | Test Acc: 0.43%
Epoch: 10 | Loss: 0.63853 | Acc: 0.69% | Test Loss: 0.59867 | Test Acc: 0.62%
Epoch: 20 | Loss: 0.44436 | Acc: 0.77% | Test Loss: 0.43514 | Test Acc: 0.71%
Epoch: 30 | Loss: 0.36791 | Acc: 0.90% | Test Loss: 0.37061 | Test Acc: 0.84%
Epoch: 40 | Loss: 0.29534 | Acc: 0.95% | Test Loss: 0.29719 | Test Acc: 0.96%
Epoch: 50 | Loss: 0.34644 | Acc: 0.81% | Test Loss: 0.50402 | Test Acc: 0.68%
Epoch: 60 | Loss: 0.11277 | Acc: 0.97% | Test Loss: 0.10818 | Test Acc: 0.97%
Epoch: 70 | Loss: 0.08050 | Acc: 0.98% | Test Loss: 0.08387 | Test Acc: 0.99%
Epoch: 80 | Loss: 0.06674 | Acc: 0.99% | Test Loss: 0.07428 | Test Acc: 0.98%
Epoch: 90 | Loss: 0.05857 | Acc: 0.99% | Test Loss: 0.06938 | Test Acc: 0.98%
